In [1]:
from pathlib import Path
import os
from IPython.display import HTML
import wandb
from mp_transformer.models.transformer import MovementPrimitiveTransformer
from mp_transformer.config import CONFIG
from mp_transformer.train import setup
from mp_transformer.utils import save_side_by_side_video

In [2]:
current_dir = Path.cwd().parts[-1]
if current_dir == "demo":
    os.chdir("..")
!pwd

/data/daniel/git/mp-transformer


In [36]:
run = wandb.init(project="mp-transformer")
artifact = run.use_artifact("tcs-mr/mp-transformer/model:v16", type='model')
artifact_dir = artifact.download()


wandb: Downloading large artifact model:v16, 87.04MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:4.2


In [30]:
# print(artifact_dir)

In [37]:
CONFIG["hidden_dim"] = 256
model, train_dataset, val_dataset = setup(CONFIG)
artifact_dir = "./artifacts/model:v16"
model = model.load_from_checkpoint(Path(artifact_dir, "model.ckpt"), config=CONFIG)
run.finish()

In [38]:
item = val_dataset[-1]
# item = val_dataset[64]
# item = val_dataset[0]
# save_side_by_side_video(item, model, from_idx=3)
save_side_by_side_video(item, model)

Video saved to tmp/comp_vid.mp4


In [39]:

HTML("""
<video width="320" height="240" controls>
  <source src="../tmp/comp_vid.mp4" type="video/mp4">
</video>
""")

In [40]:
save_side_by_side_video(item, model, from_idx=0, path="tmp/gen_vid.mp4")

poses range: [0.008441172540187836, 0.9904189705848694]
mus range: [-3.7364816665649414, 3.158311367034912]
average mu: 0.006335526704788208
logvars range: [-8.48350715637207, -0.03361109644174576]
median logvar: -0.22069960832595825
gt_latents range: [-3.1754298210144043, 3.507385492324829]
average gt_latents: 0.0032201323192566633
random_latents range: [-3.7588908672332764, 3.1629674434661865]
average random_latents: -0.0007751528173685074
Video saved to tmp/gen_vid.mp4


In [41]:


HTML("""
<video width="320" height="240" controls>
  <source src="../tmp/gen_vid.mp4" type="video/mp4">
</video>
""")